In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import gc

In [2]:
#products = pd.read_pickle('products')
#priors = pd.read_pickle('priors')
#users = pd.read_pickle('users')
#userXproduct = pd.read_pickle('userXproduct')
df_temp_new = pd.read_pickle('df_temp_new')
df_train_new = pd.read_pickle('df_train_new')
print(df_train_new.columns)
        

Index(['order_id', 'product_id', 'user_total_orders', 'user_total_items',
       'total_distinct_items', 'user_average_days_between_orders',
       'user_average_basket', 'percent_all_reordered', 'user_reorder_ratio',
       'dow', 'order_hour_of_day', 'order_hour_of_week',
       'days_since_prior_order', 'days_since_ratio', 'aisle_id',
       'department_id', 'product_orders', 'product_reorders',
       'product_reorder_rate', 'department_orders', 'department_reorders',
       'department_reorder_rate', 'aisle_orders', 'aisle_reorders',
       'aisle_reorder_rate', 'is_Organic', 'prod_average_pos_in_cart',
       'UP_orders', 'UP_reorders', 'UP_orders_ratio', 'UP_average_pos_in_cart',
       'UP_reorder_rate', 'UP_orders_since_last', 'UP_last_order_number',
       'UP_first_order_number', 'UP_order_rate_since_first_order',
       'UP_delta_hour_vs_last', 'UP_distance_radio_since_last_order',
       'UP_in_last_order', 'UP_same_dow_as_last_order', 'labels'],
      dtype='object')


In [3]:
from numpy import *
import time  
import scipy
import matplotlib.pyplot as plt  
import pandas as pd
  
  
# calculate Euclidean distance  
def euclDistance(vector1, vector2):  
    return sqrt(sum(power(vector2 - vector1, 2)))  
  
# init centroids with random samples  
def initCentroids(dataSet, k):  
    numSamples, dim = dataSet.shape  
    centroids = zeros((k, dim))  
    for i in range(k):  
        index = int(random.uniform(0, numSamples))  
        centroids[i, :] = dataSet[index, :]  
    return centroids  
  
# k-means cluster  
def kmeans(dataSet, k):  
    numSamples = dataSet.shape[0]  
    print (numSamples)
    # first column stores which cluster this sample belongs to,  
    # second column stores the error between this sample and its centroid  
    clusterAssment = mat(zeros((numSamples, 2)))  
    clusterChanged = True  
  
    ## step 1: init centroids  
    centroids = initCentroids(dataSet, k)  
  
    while clusterChanged:  
        clusterChanged = False  
        ## for each sample  
        for i in range(numSamples):  
            minDist  = 100000.0  
            minIndex = 0  
            ## for each centroid  
            ## step 2: find the centroid who is closest  
            for j in range(k):  
                distance = euclDistance(centroids[j, :], dataSet[i, :])  
                if distance < minDist:  
                    minDist  = distance  
                    minIndex = j  
              
            ## step 3: update its cluster  
            if clusterAssment[i, 0] != minIndex:  
                clusterChanged = True  
                clusterAssment[i, :] = minIndex, minDist**2  
  
        ## step 4: update centroids  
        for j in range(k):  
            pointsInCluster = dataSet[nonzero(clusterAssment[:, 0].A == j)[0]]  
            centroids[j, :] = mean(pointsInCluster, axis = 0)  
   
    return centroids, clusterAssment

df_train_dic = {}
for row in df_train_new.itertuples():
    if row.order_id not in df_train_dic.keys():
        df_train_dic[row.order_id] = [row.user_average_basket,row.total_distinct_items]
    else:
        continue
    
df_test_new = pd.read_pickle('df_test_new')
df_test_dic = {}
for row in df_test_new.itertuples():
    if row.order_id not in df_test_dic.keys():
        df_test_dic[row.order_id] = [row.user_average_basket,row.total_distinct_items]
    else:
        continue
kmeans_list = []
for keys in df_train_dic.keys():
    tmp = [keys] + df_train_dic[keys]
    kmeans_list.append(tmp)
for keys in df_test_dic.keys():
    tmp = [keys] + df_test_dic[keys]
    kmeans_list.append(tmp)
kmeans_list_in = []
for i in range(len(kmeans_list)):
    kmeans_list_in.append(kmeans_list[i][1:])

kmeans_list_in = mat(kmeans_list_in)
k = 2
centroids, clusterAssment = kmeans(kmeans_list_in, k) 
clusterAssment = clusterAssment.tolist()
kmeans_out = {}
for i in range(len(kmeans_list)):
    kmeans_out[kmeans_list[i][0]] = int(clusterAssment[i][0])

print(centroids)
print(kmeans_out[1187899])
del df_train_dic
del df_test_dic
del kmeans_list
del kmeans_list_in
del clusterAssment
del df_test_new
gc.collect()
#data = pd.DataFrame(clusterAssment)
#a = data[0].tolist()
#print (a)

206209
[[  13.79774761  155.47648621]
 [   7.60240602   41.61329651]]
1


27

In [4]:
labels = np.array(df_train_new['labels'],dtype=pd.Series)
df_train_new.drop(['labels'],axis=1,inplace=True)

f_to_use = ['user_total_orders', 'user_total_items',
       'total_distinct_items', 'user_average_days_between_orders',
       'user_average_basket', 'percent_all_reordered', 'user_reorder_ratio',
       'dow', 'order_hour_of_day', 'order_hour_of_week',
       'days_since_prior_order', 'days_since_ratio', 'aisle_id',
       'department_id', 'product_orders', 'product_reorders',
       'product_reorder_rate', 'department_orders', 'department_reorders',
       'department_reorder_rate', 'aisle_orders', 'aisle_reorders',
       'aisle_reorder_rate', 'is_Organic', 'prod_average_pos_in_cart',
       'UP_orders', 'UP_reorders', 'UP_orders_ratio', 'UP_average_pos_in_cart',
       'UP_reorder_rate', 'UP_orders_since_last', 'UP_last_order_number',
       'UP_first_order_number', 'UP_order_rate_since_first_order',
       'UP_delta_hour_vs_last', 'UP_distance_radio_since_last_order',
       'UP_in_last_order', 'UP_same_dow_as_last_order'] # 'dow', 'UP_same_dow_as_last_order'
'''
f_to_use = ['user_total_orders', 'user_average_days_between_orders', 
       'order_hour_of_day', 'days_since_prior_order',
       'aisle_id', 'department_id', 'product_orders', 'product_reorders',
       'UP_orders', 'UP_orders_ratio', 'UP_reorder_rate', 'UP_orders_since_last',
       'UP_delta_hour_vs_last'] # 'dow', 'UP_same_dow_as_last_order'
'''

"\nf_to_use = ['user_total_orders', 'user_average_days_between_orders', \n       'order_hour_of_day', 'days_since_prior_order',\n       'aisle_id', 'department_id', 'product_orders', 'product_reorders',\n       'UP_orders', 'UP_orders_ratio', 'UP_reorder_rate', 'UP_orders_since_last',\n       'UP_delta_hour_vs_last'] # 'dow', 'UP_same_dow_as_last_order'\n"

In [5]:
def eval_fun(labels, preds):
    labels = labels.split(' ')
    preds = preds.split(' ')
    rr = (np.intersect1d(labels, preds))
    precision = np.float(len(rr)) / len(preds)
    recall = np.float(len(rr)) / len(labels)
    try:
        f1 = 2 * precision * recall / (precision + recall)
    except ZeroDivisionError:
        return (precision, recall, 0.0)
    return (precision, recall, f1)

def get_pred_results(df_test,thrshold):
    d = dict()
    #TRESHOLD = thrshold
    for row in df_test.itertuples():
        TRESHOLD = thrshold[kmeans_out[row.order_id]]                
        if row.pred > TRESHOLD:
            try:
                d[row.order_id] += ' ' + str(row.product_id)
            except:
                d[row.order_id] = str(row.product_id)
    for order in df_test.order_id:
        if order not in d:
            d[order] = 'None'

    sub = pd.DataFrame.from_dict(d, orient='index')
    sub.reset_index(inplace=True)
    sub.columns = ['order_id', 'products']
    return sub


def fscore(df,bst,alpha):
    df['pred'] = bst.predict(df[f_to_use])
    train_pred=get_pred_results(df,thrshold=alpha)
    #合表
    train_pred1=pd.merge(train_pred,df_temp_new,on=['order_id'])
    #求F1结果表
    res = list()
    for entry in train_pred1.itertuples():
        res.append(eval_fun(entry[2], entry[3]))
    res = pd.DataFrame(np.array(res), columns=['precision', 'recall', 'f1'])
    return res['f1'].mean()

In [7]:
import warnings
warnings.filterwarnings('ignore')
#先使用最简单的k-Fold
from sklearn.model_selection import KFold
from lightgbm import LGBMClassifier
thrshold_list = [[0.15,0.17]]
for i in range(len(thrshold_list)):
    thrshold_now = thrshold_list[i]
    params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': {'binary_logloss'},
        'num_leaves': 96,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.95,
        'bagging_freq': 5
    }
    ROUNDS = 98
    kf=KFold(n_splits=3)    # 定义分成几个组
    list_f1=[]
    list_f2=[]
    num=1
    #clf=LGBMClassifier(objective='binary', boosting_type='gbdt')
    #决定采用手动cv  因为需要了利用合表才能得到F1 传统方法不可以 数组合表 太可怕···
    #把数组变成切边的形式  即可
    import timeit
    start=timeit.default_timer()

    for train_index,test_index in kf.split(df_train_new, labels):

        #train_max=train_index.max()
        #train_min=train_index.min()
        test_max=test_index.max()+1
        test_min=test_index.min()
        X_test=df_train_new[test_min:test_max]
        X_train=df_train_new.drop(test_index)
        #X_train,X_test=data_train[train_index],data_train[test_index]
        y_train,y_test=labels[train_index],labels[test_index]   
        d_train = lgb.Dataset(X_train[f_to_use],
                          label=y_train,
                          categorical_feature=['aisle_id', 'department_id'])  # , 'order_hour_of_day', 'dow'
        bst = lgb.train(params, d_train, ROUNDS)
        a=fscore(X_train,bst,thrshold_now)
        b=fscore(X_test,bst,thrshold_now)
        list_f1.append(a)
        list_f2.append(b)
        print('* {}: train:{}, test:{}'.format(num,a,b))
        num+=1
    print (thrshold_now)
    print('ALL:train:{} test:{}'.format(np.mean(list_f1),np.mean(list_f2)))
    end = timeit.default_timer()
    print('cost time:'+str(end-start))

    del d_train
    del X_test
    del X_train
    del y_train
    del y_test
    gc.collect()

* 1: train:0.39481814075187593, test:0.3949530602238411
* 2: train:0.3966529824829051, test:0.39180402850745993
* 3: train:0.39674947999662047, test:0.39181243661259685
[0.15, 0.17]
ALL:train:0.39607353441046717 test:0.392856508447966
cost time:934.751403755974


In [9]:
df_test_new = pd.read_pickle('df_test_new')

#clf.fit(df_train[f_to_use],labels)
d_train = lgb.Dataset(df_train_new[f_to_use],label=labels,categorical_feature=['aisle_id', 'department_id'])
#d_d=lgb.Dataset(df_test[f_to_use],categorical_feature=['aisle_id', 'department_id'])   

bst = lgb.train(params, d_train, ROUNDS)
df_test_new['pred'] = bst.predict(df_test_new[f_to_use])

sub=get_pred_results(df_test_new,[0.15,0.17])
#sub.to_csv('sub.csv', index=False)
from datetime import datetime
now = datetime.now()
sub.to_csv('lgb_results_{}.{}.{}.csv'.format(
    str(now.date()),
    str(now.hour),
    str(now.minute)
), index = False)


In [ ]:
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.cross_validation import train_test_split
from pyfm import pylibfm

from sklearn.datasets import make_classification

df_train_use = df_train[f_to_use]
train_key = df_train_use.columns.tolist()

train_list = []
for row in df_train_use.itertuples():
    tmp = {}
    for i in range(len(train_key)):
        tmp[train_key[i]] = row[i+1]
    train_list.append(tmp)
print('del df_train_use')
del df_train_use
del df_train
gc.collect()

v = DictVectorizer()
X = v.fit_transform(train_list)
print('del train_list')
del train_list
gc.collect()

y = labels
fm = pylibfm.FM(num_factors=50, num_iter=10, verbose=True, task="classification", initial_learning_rate=0.0001, learning_rate_schedule="optimal")
fm.fit(X,y)

df_test = pd.read_pickle('df_test')
df_test_use = df_test[f_to_use]
test_list = []
for row in df_test_use.itertuples():
    tmp = {}
    for i in range(len(train_key)):
        tmp[train_key[i]] = row[i+1]
    test_list.append(tmp)
del df_test_use
gc.collect()
X_test = v.transform(test_list)
del test_list
gc.collect()
pp = fm.predict(X_test)
df_test['pred'] = pp
sub=get_pred_results(df_test,0.22)
#sub.to_csv('sub.csv', index=False)
from datetime import datetime
now = datetime.now()
sub.to_csv('fm_results_{}.{}.{}.csv'.format(
    str(now.date()),
    str(now.hour),
    str(now.minute)
), index = False)

'''
train = [
    {"user": "1", "item": "5", "age": 19},
    {"user": "2", "item": "43", "age": 33},
    {"user": "3", "item": "20", "age": 55},
    {"user": "4", "item": "10", "age": 20},
]
train1 = [
    {"user": 1, "item": 5, "age": 19},
    {"user": 2, "item": 43, "age": 33},
    {"user": 3, "item": 20, "age": 55},
    {"user": 4, "item": 10, "age": 20},
]
v = DictVectorizer()
X = v.fit_transform(train)
X1 = v.fit_transform(train1)
print(X)
print(X.toarray())
print(X1)
print(X1.toarray())
y = np.repeat(1.0,X.shape[0])
print(y)
fm = pylibfm.FM(num_factors=50, num_iter=100, verbose=True, task="classification", initial_learning_rate=0.0001, learning_rate_schedule="optimal")
fm.fit(X,y)
print(v.transform({"user": 1, "item": 10, "age": 24}))
a = fm.predict(v.transform({"user": "1", "item": "10", "age": 24}))
print(a)
'''

/Users/shisaijiang/anaconda/envs/python36/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


del df_train_use
del train_list
Creating validation dataset of 0.01 of training for adaptive regularization
